In [3]:
from konlpy.tag import Okt
from gensim import corpora
import re

# 1. 파일 읽기
with open("KakaoTalk_20250511_1625_29_658_유정유정.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# 2. 메시지 추출 (날짜/이름/시간 제거)
# 형식 예시: [김소연] [오후 3:36] 정산하기를 요청했어요.
messages = re.findall(r'\[.+?\] \[.+?\] (.+)', raw_text)

# 3. 전처리 함수 정의  
okt = Okt()

def preprocess(text):
    # 특수문자 제거
    text = re.sub(r'[^가-힣\s]', '', text)
    # 명사만 추출
    nouns = okt.nouns(text)
    # 길이 1 이하 제거
    return [word for word in nouns if len(word) > 1]

# 4. 전체 문서 리스트 생성
tokenized_docs = [preprocess(msg) for msg in messages if msg.strip()]

# 5. 딕셔너리 및 코퍼스 생성
dictionary = corpora.Dictionary(tokenized_docs)
corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

# 전처리 결과 확인
print(tokenized_docs[:3])

SystemError: java.lang.UnsatisfiedLinkError: C:\Users: Can't find dependent libraries